In [8]:
from keras.preprocessing import image
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.models import Model, Sequential, save_model, load_model
from keras.layers import Dense, GlobalAveragePooling2D, Flatten, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
import numpy as np
from sklearn.metrics import classification_report
from keras.utils import to_categorical

In [2]:
train_dataset = 'train/'
test_dataset = 'test/'

# Configuração do gerador de dados para pré-processamento e aumento de dados
datagen = ImageDataGenerator(
    rescale=1./255,         # Normaliza os valores dos pixels para o intervalo [0, 1]
    #shear_range=0.2,        # Aplica corte aleatório
    #zoom_range=0.2,         # Aplica zoom aleatório
    #horizontal_flip=True,   # Inverte aleatoriamente as imagens horizontalmente
)

# Carrega e pré-processa as imagens de treinamento
train_generator = datagen.flow_from_directory(
    train_dataset,
    target_size=(224, 224),  # Tamanho das imagens esperado pela InceptionV3
    batch_size=32,
    class_mode='categorical', # 'categorical' se for classificação com mais de duas classes
    #subset='training'        # Especifica se é para conjunto de treinamento
)

# Carrega e pré-processa as imagens de validação
test_generator = datagen.flow_from_directory(
    test_dataset,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    #subset='validation'      # Especifica se é para conjunto de validação
)

Found 18325 images belonging to 10 classes.
Found 7854 images belonging to 10 classes.


In [3]:
model = Sequential()

model.add(
    InceptionV3(weights='imagenet', input_shape=(224, 224, 3), include_top=False)
)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

# Cria um novo modelo com as camadas personalizadas
model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

# y_train_one_hot = to_categorical(train_generator.labels, num_classes=10)

# Substitua X_train e y_train pelos seus dados de treinamento e rótulos
history = model.fit(train_generator, epochs=5, batch_size=32, validation_data=test_generator)

Epoch 1/5
573/573 [==============================] - 2224s 4s/step - loss: 0.4060 - accuracy: 0.8843 - val_loss: 0.2150 - val_accuracy: 0.9402
Epoch 2/5
573/573 [==============================] - 2203s 4s/step - loss: 0.1851 - accuracy: 0.9507 - val_loss: 0.1944 - val_accuracy: 0.9450
Epoch 3/5
573/573 [==============================] - 2192s 4s/step - loss: 0.1224 - accuracy: 0.9652 - val_loss: 0.2404 - val_accuracy: 0.9391
Epoch 4/5
573/573 [==============================] - 2304s 4s/step - loss: 0.1467 - accuracy: 0.9646 - val_loss: 0.1979 - val_accuracy: 0.9491
Epoch 5/5
573/573 [==============================] - 2190s 4s/step - loss: 0.0639 - accuracy: 0.9819 - val_loss: 0.2247 - val_accuracy: 0.9487


In [9]:
model.save('modelo.h5')

c:\Users\gabri\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [10]:
loaded_model = load_model('modelo.h5')

In [11]:
predictions = model.predict(test_generator)

246/246 [==============================] - 137s 555ms/step


In [5]:
target_names = ['Aranha', 'Borboleta', 'Cachorro', 'Cavalo', 'Elefante', 'Esquilo', 'Galinha', 'Gato', 'Ovelha', 'Vaca']
test_generator.reset()
predictions = model.predict(test_generator)
y_pred = np.round(predictions).astype(int)
y_pred.head()
y_true = test_generator.labels
report = classification_report(y_true, y_pred, target_names=target_names)
print(report)

246/246 [==============================] - 134s 544ms/step


AttributeError: 'numpy.ndarray' object has no attribute 'head'